In [14]:
#간단 테스팅용입니다
#json파일로 만들거는 나중에 age_gender_predicted_json.ipynb 형태로 할 예정

In [1]:
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import json
from sklearn.metrics import mean_absolute_error, accuracy_score
from facenet_pytorch import MTCNN


c:\Users\yango\anaconda3\envs\eunseo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(torch.__version__)
print(torch.cuda.is_available())  # True면 GPU 인식됨

2.2.2+cu121
True


In [4]:
# Data Processing class 만들기 파이프라인... 좀 정리하고 싶은데 일단 적읍시다

class DataProcessing(Dataset) :
    
    #생성자 생성
    def __init__(self, image_dir, label_dir, categories, transform=None, emotion_return=False,
                 mode='train', augment_transform=None, mtcnn=None) :
        
        self.datalist = [] #이제 전체데이터 쓸거라 data로 변수명 바꿈
        self.transform = transform
        self.augment_transform = augment_transform
        self.mtcnn = mtcnn
        self.label_map = { cat :  idx for idx, cat in enumerate(categories)}
        self.emotion_return = emotion_return
        self.mode=mode
        self.age_min = 10
        self.age_max = 60
        
        for category in categories :
            
            json_path = os.path.join(label_dir, f'{self.mode}_{category}.json')
            img_folder = os.path.join(image_dir, category)
            
            with open(json_path, 'r', encoding='euc-kr') as f :
                label_data = json.load(f)
            
            for row in label_data :
                
                filename = row['filename']
                img_path = os.path.join(img_folder, filename)
                
                if os.path.isfile(img_path) :
                    
                    age = row.get('age') 
                    
                    #60대면 skip
                    if age is not None and age >= 60 :
                        continue 
                    
                    #나이 정규화
                    if age is not None : 
                        age_norm = (age - self.age_min) / (self.age_max - self.age_min) 
                    #결측값 (있지않지만 만약을 대비)
                    else :
                        age_norm = 0.0
                        
                    data = {
                        'img_path' : img_path,
                        'category' : category,
                        'age' : age_norm,
                        'raw_age' : age,
                        'gender' : row.get('gender')
                    }
                    
                    self.datalist.append(data)
    
    
    def __len__(self) :
        
        return len(self.datalist)
    
    def __getitem__(self, idx) :
        
        data_item = self.datalist[idx]
        image = Image.open(data_item['img_path']).convert('RGB')
        
        if self.mtcnn is not None :
            
            face_img = self.mtcnn(image)
            
            if face_img is None :
                image = image.resize((224,224))
                face_img = transforms.ToTensor()(image)
                
        else :
            
            face_img = transforms.ToTensor()(image)
            
            
            
            
        age_norm = data_item['age']
        
        augment_flag = False
        if (0 <= age_norm <= 0.18) or (0.6 <= age_norm <= 0.78) or (0.8 <= age_norm <= 0.98):
            augment_flag = True

        if self.mode == 'train':
            if augment_flag and self.augment_transform is not None:
                face_img = self.augment_transform(face_img)
            elif self.transform is not None:
                face_img = self.transform(face_img)
        else:
            if self.transform is not None:
                face_img = self.transform(face_img)

        
        #emotion_label = torch.tensor(self.label_map[sample['category']], dtype=torch.long) 수정1
        age=torch.tensor(data_item['age'], dtype=torch.float32)
        gender = torch.tensor(1 if data_item['gender']=='남' else 0, dtype=torch.long)

        # return image, emotion_label, age, gender 수정1

        if self.emotion_return :
            emotion_label = torch.tensor(self.label_map[data_item['category']], dtype=torch.long)
            return face_img, emotion_label, age, gender

        else :
            return face_img, age, gender
       

In [5]:
mtcnn = MTCNN(image_size=224, margin=20, min_face_size=20,thresholds=[0.6, 0.7, 0.7], device=device)

In [6]:
transform = transforms.Compose([
    #transforms.Resize((224,224)),
    #transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [7]:
augment_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.RandomRotation(10),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [8]:
model = resnet50()
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features,256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.4),
    
    nn.Linear(256,128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.3),
    
    nn.Linear(128,64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.2),
    
    nn.Linear(64,3)  # [age(1), gender(2)] 출력
)

In [ ]:
base_dir = os.path.dirname(os.path.abspath(__file__))
weight_path = os.path.join(base_dir, 'pth_pkl', 'model_aug_weights_v2_2_1_4.pth')

model.load_state_dict(torch.load(weight_path, map_location=device))
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
categories = ['anger','happy','panic','sadness']
base_dir = os.path.dirname(os.path.abspath(__file__))
test_image_dir = os.path.join(base_dir, 'test', 'image')
test_label_dir = os.path.join(base_dir, 'test', 'label')

In [12]:
test_dataset = DataProcessing(test_image_dir, test_label_dir, categories, transform=transform, mode='test',mtcnn=mtcnn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
print(test_dataset.age_min, test_dataset.age_max)

10 60


In [14]:
# 누적할 리스트 생성
all_true_ages = []
all_pred_ages = []
all_true_genders = []
all_pred_genders = []

# 5. 추론 + 평가
with torch.no_grad():
    for images, ages, genders in test_loader:
        images = images.to(device)
        ages = ages.to(device)
        genders = genders.to(device)

        outputs = model(images)
        pred_age = outputs[:,0]
        pred_gender_logits = outputs[:,1:3]

        # 나이 역정규화
        pred_age_orig = pred_age * (test_dataset.age_max - test_dataset.age_min) + test_dataset.age_min
        true_age_orig = ages * (test_dataset.age_max - test_dataset.age_min) + test_dataset.age_min

        pred_gender = torch.argmax(pred_gender_logits, dim=1)

        # 리스트에 누적
        all_pred_ages.extend(pred_age_orig.cpu().numpy())
        all_true_ages.extend(true_age_orig.cpu().numpy())
        all_pred_genders.extend(pred_gender.cpu().numpy())
        all_true_genders.extend(genders.cpu().numpy())

        # 샘플 출력
        for i in range(len(images)):
            pred_gender_str = '남' if pred_gender[i].item() == 1 else '여'
            true_gender_str = '남' if genders[i].item() == 1 else '여'
            print(f'predicted age : {pred_age_orig[i].item():.2f}세, actual age: {true_age_orig[i].item():.2f}세')
            print(f'predicted gender: {pred_gender_str}, actual gender: {true_gender_str}')

# 전체 결과 계산
test_mae = mean_absolute_error(all_true_ages, all_pred_ages)
test_acc = accuracy_score(all_true_genders, all_pred_genders)

print('\n 테스트 평가 결과')
print(f'나이 MAE: {test_mae:.4f}')
print(f'성별 정확도: {test_acc:.4f}')

predicted age : 32.33세, actual age: 30.00세
predicted gender: 여, actual gender: 여
predicted age : 22.37세, actual age: 40.00세
predicted gender: 남, actual gender: 남
predicted age : 23.19세, actual age: 30.00세
predicted gender: 남, actual gender: 남
predicted age : 21.40세, actual age: 20.00세
predicted gender: 남, actual gender: 남
predicted age : 24.78세, actual age: 30.00세
predicted gender: 여, actual gender: 여
predicted age : 22.13세, actual age: 20.00세
predicted gender: 남, actual gender: 남
predicted age : 23.08세, actual age: 20.00세
predicted gender: 여, actual gender: 여
predicted age : 22.03세, actual age: 40.00세
predicted gender: 남, actual gender: 남
predicted age : 29.50세, actual age: 30.00세
predicted gender: 여, actual gender: 여
predicted age : 22.98세, actual age: 20.00세
predicted gender: 남, actual gender: 여
predicted age : 22.79세, actual age: 40.00세
predicted gender: 남, actual gender: 남
predicted age : 23.72세, actual age: 20.00세
predicted gender: 남, actual gender: 여
predicted age : 29.19세, actu

In [16]:
import pandas as pd

In [ ]:
# 성별 숫자 → 문자 변환 함수
def gender_to_str(gender_int):
    return '남' if gender_int == 1 else '여'

# DataFrame 만들기
import numpy as np
df = pd.DataFrame({
    '예측 나이': np.round(all_pred_ages, 2),
    '실제 나이': np.round(all_true_ages, 2),
    '예측 성별': [gender_to_str(g) for g in all_pred_genders],
    '실제 성별': [gender_to_str(g) for g in all_true_genders]
})

# CSV 저장 (utf-8-sig로 한글 깨짐 방지)
base_dir = os.path.dirname(os.path.abspath(__file__))
save_path = os.path.join(base_dir, 'test_result', 'test_v2_2_1_4.csv')

df.to_csv(save_path, index=False, encoding='utf-8-sig')
print(f"CSV 파일로 결과 저장 완료: {save_path}")

CSV 파일로 결과 저장 완료: test_v2_2_1_4.csv
